In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


data = pd.read_csv('cleaned.csv', index_col=0)

# For developing purpose
# data = data.sample(n=2000, random_state=42)

print(data.head())  


  origin finaldest  year  quarter  stops  avgprice  OriginLongitude  \
0    ABI       ABQ  2010        2      1    235.40          -99.682   
1    ABI       ABQ  2010        3      1    180.75          -99.682   
2    ABI       ABQ  2010        4      1    273.00          -99.682   
3    ABI       ABQ  2011        1      1    223.00          -99.682   
4    ABI       ABQ  2011        2      1    298.00          -99.682   

   OriginLatitude  DestLongitude  DestLatitude  
0          32.411       -106.609         35.04  
1          32.411       -106.609         35.04  
2          32.411       -106.609         35.04  
3          32.411       -106.609         35.04  
4          32.411       -106.609         35.04  


In [3]:
X = data.drop(columns=['avgprice'])  
y = data['avgprice'] 

X = data.drop(columns=['avgprice']) 
y = data['avgprice'] 

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
X_raw = pd.concat([X_train_raw, X_test_raw])

In [4]:

X = pd.get_dummies(X_raw, columns=['origin', 'finaldest'], drop_first=True)

X_train = X.iloc[:len(X_train_raw)].reset_index(drop=True)
X_test = X.iloc[len(X_train_raw):].reset_index(drop=True)

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")


Training set: (550463, 475), (550463,)
Test set: (28972, 475), (28972,)


##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

rf_model = RandomForestRegressor(random_state=42, n_estimators=10, max_depth=10)
rf_model.fit(X_train, y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Training MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")
print(f"Training MAE: {train_mae:.4f}")
print(f"Test MAE: {test_mae:.4f}")



##XGB

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(tree_method='hist', device='cuda', n_estimators=10, max_depth=10, random_state=42)

xgb_model.fit(X_train, y_train)

y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Training MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")
print(f"Training MAE: {train_mae:.4f}")
print(f"Test MAE: {test_mae:.4f}")


d:\Tool\Anaconda\envs\10703\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:08:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Training MSE: 18795.0833
Test MSE: 23950.8395
Training MAE: 89.5970
Test MAE: 95.6015


##Simple GCN

In [37]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np

class GCN(torch.nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, global_feature_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(node_feature_dim, 128)
        self.conv2 = GCNConv(128, 32)
        self.conv3 = GCNConv(32, 8)
        self.fc1 = torch.nn.Linear(8*2 + edge_feature_dim + global_feature_dim, 16)
        self.fc2 = torch.nn.Linear(16, 1) 


    def forward(self, x, edge_index, edge_attr, year, quarter):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        
        x = torch.cat([x[edge_index[0]], x[edge_index[1]], edge_attr, year, quarter], dim=1)
        
        x = F.relu(self.fc1(x))
        return self.fc2(x).squeeze()

In [38]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool, global_max_pool

class LargeGCN(torch.nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, global_feature_dim):
        super(LargeGCN, self).__init__()

        # GCN Layers (Wider feature sizes)
        self.conv1 = GCNConv(node_feature_dim, 1024)  # Feature size: 1024
        self.conv2 = GCNConv(1024, 512)              # Feature size: 512
        self.conv3 = GCNConv(512, 256)               # Feature size: 256
        self.conv4 = GCNConv(256, 128)               # Feature size: 128
        self.conv5 = GCNConv(128, 64)                # Feature size: 64
        
        # Attention Mechanism (Enhances feature importance)
        self.attn1 = GATConv(64, 64, heads=4, concat=True)
        self.attn2 = GATConv(256, 64, heads=4, concat=True)

        # Edge Features Transformation
        self.edge_fc = torch.nn.Linear(edge_feature_dim, 128)

        # Global Features Transformation
        self.global_fc = torch.nn.Linear(global_feature_dim, 128)

        # Graph Pooling Layers (Global Summary)
        self.global_pool_mean = global_mean_pool
        self.global_pool_max = global_max_pool

        # Fully Connected Layers (Post-pooling)
        self.fc1 = torch.nn.Linear(64 * 4 + 128 + 128, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.fc3 = torch.nn.Linear(256, 1)

        # Regularization
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x, edge_index, edge_attr, batch, year, quarter):
        # GCN Layers with ReLU activation
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))

        # Attention Mechanism
        x = F.relu(self.attn1(x, edge_index))
        x = F.relu(self.attn2(x, edge_index))

        # Edge features
        edge_attr = F.relu(self.edge_fc(edge_attr))

        # Global features
        global_features = F.relu(self.global_fc(torch.cat([year, quarter], dim=1)))

        # Pooling (Summarize the graph)
        x_mean = self.global_pool_mean(x, batch)
        x_max = self.global_pool_max(x, batch)
        x = torch.cat([x_mean, x_max], dim=1)

        # Concatenate with edge and global features
        x = torch.cat([x, edge_attr, global_features], dim=1)

        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x).squeeze()


In [39]:
torch.cuda.is_available()

True

In [40]:

airport_map = {airport: idx for idx, airport in enumerate(pd.concat([X_raw['origin'], X_raw['finaldest']]).unique())}
X_raw['origin_idx'] = X_raw['origin'].map(airport_map)
X_raw['finaldest_idx'] = X_raw['finaldest'].map(airport_map)

X_train = X_raw.iloc[:len(X_train_raw)].reset_index(drop=True)
X_test = X_raw.iloc[len(X_train_raw):].reset_index(drop=True)


In [41]:
edge_index_train = torch.tensor(np.array([X_train['origin_idx'].values, X_train['finaldest_idx'].values]), dtype=torch.long)
edge_index_test = torch.tensor(np.array([X_test['origin_idx'].values, X_test['finaldest_idx'].values]), dtype=torch.long)

# for edge attribute, it currently only stores the stops.
edge_attr_train = torch.tensor(X_train['stops'].values.reshape(-1, 1), dtype=torch.float)
edge_attr_test = torch.tensor(X_test['stops'].values.reshape(-1, 1), dtype=torch.float)

y_train = torch.tensor(y_train.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)

year_train = torch.tensor(X_train['year'].values, dtype=torch.float).reshape(-1, 1)
quarter_train = torch.tensor(X_train['quarter'].values, dtype=torch.float).reshape(-1, 1)
year_test = torch.tensor(X_test['year'].values, dtype=torch.float).reshape(-1, 1)
quarter_test = torch.tensor(X_test['quarter'].values, dtype=torch.float).reshape(-1, 1)

num_nodes = len(airport_map)
node_features = torch.eye(num_nodes)


In [42]:
#Node feature: onehot code + cooardinates

coordinates = pd.DataFrame({
    'airport': list(airport_map.keys()),
    'longitude': [X_raw[X_raw['origin'] == airport]['OriginLongitude'].values[0] if airport in X_raw['origin'].values else X_raw[X_raw['finaldest'] == airport]['DestLongitude'].values[0] for airport in airport_map],
    'latitude': [X_raw[X_raw['origin'] == airport]['OriginLatitude'].values[0] if airport in X_raw['origin'].values else X_raw[X_raw['finaldest'] == airport]['DestLatitude'].values[0] for airport in airport_map]
})

one_hot_nodes = torch.eye(len(airport_map))  
coordinates_tensor = torch.tensor(coordinates[['longitude', 'latitude']].values, dtype=torch.float)
node_features = torch.cat([one_hot_nodes, coordinates_tensor], dim=1)

In [45]:
from torch.optim.lr_scheduler import StepLR

node_feature_dim = node_features.shape[1] 
edge_feature_dim = edge_attr_train.shape[1]
global_feature_dim = 2  # Year and quarter as two additional features
model = GCN(node_feature_dim, edge_feature_dim, global_feature_dim)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to()

from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=100, gamma=0.5)
criterion = torch.nn.MSELoss(reduction='mean')

def train():
    model.train()
    optimizer.zero_grad()
    output = model(node_features, edge_index_train, edge_attr_train, year_train, quarter_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    with torch.no_grad():
        output = model(node_features, edge_index_test, edge_attr_test, year_test, quarter_test)
        loss = criterion(output, y_test)
    return loss.item()

num_epochs = 1000
for epoch in range(num_epochs):
    train_loss = train()
    scheduler.step()
    
    if epoch % 10 == 0:
        test_loss = test()
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

Epoch 1/1000, Train Loss: 105125.5938, Test Loss: 106192.2656
Epoch 11/1000, Train Loss: 95084.8203, Test Loss: 96099.1094
Epoch 21/1000, Train Loss: 85682.2344, Test Loss: 86768.3828
Epoch 31/1000, Train Loss: 76260.6719, Test Loss: 77242.2578
Epoch 41/1000, Train Loss: 66217.7344, Test Loss: 67111.7188
Epoch 51/1000, Train Loss: 56021.6289, Test Loss: 56903.1875
Epoch 61/1000, Train Loss: 46890.3516, Test Loss: 47923.1719
Epoch 71/1000, Train Loss: 40782.2812, Test Loss: 42188.5391
Epoch 81/1000, Train Loss: 39024.8633, Test Loss: 40770.9766
Epoch 91/1000, Train Loss: 39080.5039, Test Loss: 40818.4414
Epoch 101/1000, Train Loss: 38978.2109, Test Loss: 40727.6133
Epoch 111/1000, Train Loss: 38930.7656, Test Loss: 40675.3984
Epoch 121/1000, Train Loss: 38871.2891, Test Loss: 40613.0195
Epoch 131/1000, Train Loss: 38810.8125, Test Loss: 40549.8984
Epoch 141/1000, Train Loss: 38749.3594, Test Loss: 40486.3203
Epoch 151/1000, Train Loss: 38693.7148, Test Loss: 40429.1953
Epoch 161/1000, T